# Test run

In [1]:
#using Pkg
using HDF5
using JSON
using SpecialFunctions
using JLD

## Get documents

In [2]:
vocab = []
texts = []

open("./data/hep-ph.json", "r") do f
    i=1
    for line in eachline(f)
        jdict = JSON.parse(line)
        words = split(jdict["summary"],r" | |\.|\,|\n|\$")
        push!(texts, words)
        union!(vocab, collect(words))
        i += 1
    end
end

vocab_size,  = size(vocab)
word_to_id = Dict()
id_to_word = Dict()
for i in 1:vocab_size
    word_to_id[vocab[i]] = i
    id_to_word[i] = vocab[i]
end

First make whole corpus

In [3]:
texts_size, = size(texts)
corpus = []
for text in texts
    words_id = map(w->word_to_id[w], text)
    
    counts_d = Dict{Int, Int}()
    for iv in words_id
        counts_d[iv] = get(counts_d, iv, 0) + 1
    end

    counts_d_tuple = []
    for (key, ct) in counts_d
        push!(counts_d_tuple, (key, ct))
    end
    push!(corpus, counts_d_tuple)
end

Then devide it into corpus for training and test.

In [4]:
# devide test and train
train_ratio = 0.8
train_corpus = []
test_corpus = []

for d in corpus
    Nd, = size(d)
    length = Int(round(Nd*0.8))
    push!(train_corpus, d[1:length])
    push!(test_corpus, d[length+1:end])
end

## Run

In [5]:
include("../src/LDA_CGS.jl")

Main.LDA_CGS

First make LDA object

In [6]:
K = 20
V = vocab_size
x = LDA_CGS.LDA(K, V)

Main.LDA_CGS.LDA(20, 13182, 1609732, #undef, #undef, #undef, #undef, #undef, #undef, 5.0e-324, 2.252070422e-314, 5.0e-323, #undef, 4558241281, 6.0e-323, #undef, #undef, #undef)

Then run

In [7]:
burnin = 10
sample = 2
LDA_CGS.run(x, train_corpus, test_corpus, burnin, sample)

Burn-in (period=10)...
epoch=1
epoch=2
epoch=3
epoch=4
epoch=5
epoch=6
epoch=7
epoch=8
epoch=9
epoch=10
Sample from the posterior...
epoch=1, PPL=46.027711925313994
epoch=2, PPL=45.37632130193404
